In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import StandardScaler

# Load the dataset
input_file = "merged_data.csv"
df = pd.read_csv(input_file)

# Remove leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Log original record count
print(f"Total records before cleaning: {df.shape[0]}")

# 1. Identifying Missing Data
missing_data = df.isnull().sum() / len(df) * 100
print("Missing Data (%):\n", missing_data)

# 3. Handling Missing Data
threshold = 50  # Drop columns with >50% missing data
df = df.dropna(axis=1, thresh=len(df) * (threshold / 100))

# Fill missing values in numerical columns with the median
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].median())

# Fill missing values in categorical columns with the mode
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# 4. Data Type Conversions
# Convert 'datetime' column to datetime format
if "datetime" in df.columns:
    df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce", utc=True)

# Ensure numerical columns are properly cast
for col in df.select_dtypes(include=["object"]).columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        df[col] = df[col].astype("category")

# 5. Handling Duplicates
duplicate_count = df.duplicated().sum()
print(f"Total duplicate records: {duplicate_count}")
df.drop_duplicates(inplace=True)
print(f"Total records after removing duplicates: {df.shape[0]}")

# 6. Feature Engineering
if "datetime" in df.columns:
    df["hour"] = df["datetime"].dt.hour
    df["day"] = df["datetime"].dt.day
    df["month"] = df["datetime"].dt.month
    df["year"] = df["datetime"].dt.year
    df["weekday"] = df["datetime"].dt.weekday
    df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)

    # Remove 'season' categorical column
    # Create separate season binary columns instead
    df["season_Winter"] = df["month"].isin([12, 1, 2]).astype(int)
    df["season_Spring"] = df["month"].isin([3, 4, 5]).astype(int)
    df["season_Summer"] = df["month"].isin([6, 7, 8]).astype(int)
    df["season_Fall"] = df["month"].isin([9, 10, 11]).astype(int)

# 7. One-Hot Encoding with Integer Conversion
def one_hot_encode(df, column_name, drop_first=False):
    """ One-hot encodes a categorical column and converts new dummy columns to int. """
    if column_name in df.columns:
        df = pd.get_dummies(df, columns=[column_name], drop_first=drop_first)  # Drop first category if needed
        dummy_cols = [col for col in df.columns if column_name in col]  # Find newly created columns
        df[dummy_cols] = df[dummy_cols].astype(int)  # Convert to integers
    return df

# Apply encoding to categorical features
df = one_hot_encode(df, "Province", drop_first=True)  # Avoid dummy variable trap

# 8. Convert "subba-name" into True/False Features (Binary Columns)
if "subba-name" in df.columns:
    subba_dummies = pd.get_dummies(df["subba-name"], prefix="subba", dtype=int)
    df = pd.concat([df, subba_dummies], axis=1)
    df.drop(columns=["subba-name"], inplace=True)  # Drop original column

# 9. Convert "weather_name" into 4 Separate Binary Columns (No Drop)
if "weather_name" in df.columns:
    weather_dummies = pd.get_dummies(df["weather_name"], prefix="weather", dtype=int)
    df = pd.concat([df, weather_dummies], axis=1)
    df.drop(columns=["weather_name"], inplace=True)  # Drop original column

# 10. Normalizing Numerical Features (Optional)
# scaler = StandardScaler()
# numerical_cols = df.select_dtypes(include=[np.number]).columns
# df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Save cleaned data
output_csv = "cleaned_data.csv"
df.to_csv(output_csv, index=False)
print(f"Cleaned data saved as {output_csv}")


Total records before cleaning: 956351
Missing Data (%):
 subba             0.0
subba-name        0.0
parent            0.0
parent-name       0.0
value             0.0
value-units       0.0
datetime          0.0
temperature_2m    0.0
dtype: float64
Total duplicate records: 666820
Total records after removing duplicates: 289531
Cleaned data saved as cleaned_data.csv
